In [1]:
import alephclient
import os

# Testing on the live instance
This notebook has no scientific purpose, but ease testing on the live instance of Aleph.

In [2]:
path = "data/testWorkflow"

In [3]:
host="http://localhost:3000"
api_key="lUuPrgpmvqM24Mktqbtefw8cmPGY9U4ky0eotN44Kbc" 
coll_1 = "3b41686198c643e1913eafe2aed108d1"#"ce6318ac176844bf90410c83d7e1cd87"
coll_2 = "edf9766479254e4fafbd2a3114774dec"#"a137123a9cf04445892a6fb127ba4013"
target_collection_id = "4b713bcbe372492894529764d7f8096e"
unlabled = "./data/meineabgeordneten_agg.json"

In [4]:
collpath_1 = path + "/coll1.json"
collpath_2 = path + "/coll2.json"

In [ ]:
%%bash
pip install ./merger

# Load Collections From Aleph


Collection 1

In [14]:
%%bash -s "$host" "$api_key" "$coll_1" "$collpath_1" 
alephclient --host $1 --api-key $2 stream-entities -f $3 | ftm aggregate -o $4

Collection 2

In [15]:
%%bash -s "$host" "$api_key" "$coll_2" "$collpath_2" 
alephclient --host $1 --api-key $2 stream-entities -f $3 | ftm aggregate -o $4

# Enrich

In [19]:
enrich_path = path + "/enrich.json"

In [20]:
%%bash -s "$path" "$enrich_path"
merger extract -i $1 -p wikidataId | merger enrich -o $2

Item with id 'Q12345' not matchable to FtM
/home/peter/anaconda3/envs/bachelor_thesis/lib/python3.9/site-packages/countrynames/__init__.py:21: ICUWarning: Install 'pyicu' for better text transliteration.
  return normalize(country, latinize=True)
Item with id 'Q83822' not matchable to FtM


# Merge

In [21]:
cli_merged = "data/testWorkflow/mgd/merged.json"

In [22]:
%%bash -s "$path" "$cli_merged"
merger pmatch -i $1 | merger pmerge  -i $1 | ftm aggregate -o $2 

# Upload

In [19]:
%%bash -s "$host" "$api_key" "$target_collection_id" "$cli_merged" 
cat $4 | alephclient --host $1 --api-key $2 write-entities -f $3

# Delete Target Collection

In [10]:
%%bash -s "$host" "$api_key" "$target_collection_id"
alephclient --host $1 --api-key $2 flush -f $3